In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier

In [2]:
df = pd.read_csv("train.csv")
df.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


In [3]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [4]:
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)
df.head(3)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S


### Splitting the Dataset :

In [5]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['Survived']),
                                                    df['Survived'],
                                                    test_size=0.2,
                                                    random_state=42)

In [6]:
X_train.head(3)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.500,S
733,2,male,23.0,0,0,13.000,S
382,3,male,32.0,0,0,7.925,S


In [7]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(712, 7)
(179, 7)
(712,)
(179,)


### Applying Imputation :

In [8]:
# Default Strategy => 'mean'
si_age = SimpleImputer()  
si_embarked = SimpleImputer(strategy='most_frequent')

X_train_age = si_age.fit_transform(X_train[['Age']])
X_train_embarked = si_embarked.fit_transform(X_train[['Embarked']])

X_test_age = si_age.transform(X_test[['Age']])
X_test_embarked = si_embarked.transform(X_test[['Embarked']])

In [9]:
print(X_train_age.shape)
print(X_train_embarked.shape)

(712, 1)
(712, 1)


### One-Hot Encoding :

In [10]:
ohe_sex = OneHotEncoder(sparse_output=False , handle_unknown='ignore')
ohe_embarked = OneHotEncoder(sparse_output=False , handle_unknown='ignore')

X_train_sex = ohe_sex.fit_transform(X_train[['Sex']])
X_train_embarked = ohe_embarked.fit_transform(X_train_embarked)

X_test_sex = ohe_sex.transform(X_test[['Sex']])
X_test_embarked = ohe_embarked.transform(X_test_embarked)


In [11]:
X_train_sex

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [0., 1.],
       [1., 0.],
       [0., 1.]])

In [12]:
X_train_embarked

array([[0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]])

In [13]:
print(X_train_sex.shape)
print(X_train_embarked.shape)

(712, 2)
(712, 3)


In [14]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S


In [15]:
# Extracting the unchanged columns
X_train_rem  = X_train.drop(columns=['Sex','Age','Embarked'])
X_test_rem  = X_test.drop(columns=['Sex','Age','Embarked'])

In [16]:
# Joining the Arrays 
X_train_transformed = np.concatenate((X_train_rem, X_train_age ,X_train_sex ,X_train_embarked),axis=1)
X_test_transformed = np.concatenate((X_test_rem, X_test_age ,X_test_sex ,X_test_embarked),axis=1)

In [17]:
print(X_train_transformed.shape)
print(X_test_transformed.shape)

(712, 10)
(179, 10)


### Building Model :

In [18]:
clf = DecisionTreeClassifier()
clf.fit(X_train_transformed , y_train)

DecisionTreeClassifier()

In [19]:
y_pred = clf.predict(X_test_transformed)

In [26]:
from sklearn.metrics import accuracy_score

In [27]:
print("Accuracy : ",accuracy_score(y_test,y_pred))

Accuracy :  0.776536312849162


### Using Pickle :

In [28]:
import pickle

In [30]:
pickle.dump(ohe_sex, open('C:/Users/gupta/OneDrive/Documents/Machine Learning/14_Pipelines/ohe_sex.pkl','wb'))
pickle.dump(ohe_embarked, open('C:/Users/gupta/OneDrive/Documents/Machine Learning/14_Pipelines/ohe_embarked.pkl','wb'))
pickle.dump(clf, open('C:/Users/gupta/OneDrive/Documents/Machine Learning/14_Pipelines/clf.pkl','wb'))
